# GPT, Junior Developer

Let's see if we can get GPT to fix bugs.

# Init

In [44]:
import logging
import os
import sys

from dotenv import load_dotenv

# for jupyter notebooks
import nest_asyncio
from IPython.display import display, Markdown
###

from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import Document, GPTListIndex

from alchemy.loader import ModelOptions, RepoOptions, CacheOptions

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()

assert (os.environ["GITHUB_TOKEN"] and os.environ["OPENAI_API_KEY"])

nest_asyncio.apply()

# common options
repo_opts = RepoOptions(owner="inhumantsar", repo="tacostats")
# setting this to True forces a cache update on every run.
# useful for testing but it's not free! watch your token usage.
cache_opts = CacheOptions(force_update=True)

# primer data for the chat model
# PREHISTORY = [
#     "You are a software developer helping to refactor, improve, and maintain a codebase.",
#     "When prompted to perform an update, you must return the entire updated file.",
#     "Format your responses as plaintext without markdown, as if you were editing the file directly.",
# ]
# splitter = TokenTextSplitter(separator=" ", chunk_size=2048, chunk_overlap=20)
# prehistory_chunks = [splitter.split_text(prehistory) for prehistory in PREHISTORY]
#
# ... i thought loading this into the memory index would work, but apparently it does stuff in the translating
# subclasses into the base class before storing or feeding them into the agent. it does an isinstance check
# and fails. so i'm just going to use a prompt template instead...
PROMPT_TEMPLATE = """
You are a software developer helping to refactor, improve, and maintain a codebase. When prompted to 
perform an update, you must return the entire updated file. Format your responses as plaintext without 
markdown, as if you were editing the file directly.
{chat_history}
Human: {question}
AI:
"""

# the "github issue" we're going to simulate
PROMPT = "Update the memory size for all harvester-related functions in lambda.template.yaml to 512mb."



# gpt-3.5-turbo

## Options

In [9]:
# this goes into the service context and it seems like this cannot be changed after the index is created
gpt35t_model_opts = ModelOptions(model_name="gpt-3.5-turbo", temperature=0.2)


## Create index

In [10]:
from alchemy.loader import create_simple_vector_index
gpt35t_index = create_simple_vector_index(repo_opts, cache_opts, gpt35t_model_opts)

INFO:alchemy.loader:loading simple vector index...


loading simple vector index...
loading simple vector index...


INFO:alchemy.loader:repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}


repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}


INFO:alchemy.loader:cache_opts: {'path': '~/.alchemy', 'force_update': True, 'max_age': 60}


...

Unknown max input size for gpt-3.5-turbo, using defaults.
Unknown max input size for gpt-3.5-turbo, using defaults.
current path: 
tree data: GitTreeResponseModel(sha='9158e2d024c169d29eb94fdd327b724402381cb0', url='https://api.github.com/repos/inhumantsar/tacostats/git/trees/9158e2d024c169d29eb94fdd327b724402381cb0', tree=[GitTreeResponseModel.GitTreeObject(path='.bumpversion.cfg', mode='100644', type='blob', sha='e17f6921b0bc7604766dd58489eda60652564cc0', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/e17f6921b0bc7604766dd58489eda60652564cc0', size=199), GitTreeResponseModel.GitTreeObject(path='.dockerignore', mode='100644', type='blob', sha='f0ef4d8ac83ceb8113228e39d491d8043df4da22', url='https://api.github.com/repos/inhumantsar/tacostats/git/blobs/f0ef4d8ac83ceb8113228e39d491d8043df4da22', size=75), GitTreeResponseModel.GitTreeObject(path='.gitignore', mode='100644', type='blob', sha='fe1c09354be3309cbe42fc37d3c9818a933bde11', url='https://api.github.com/repos/in

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens


> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 32893 tokens


> [build_index_from_nodes] Total embedding token usage: 32893 tokens
> [build_index_from_nodes] Total embedding token usage: 32893 tokens


## Simulate a GitHub issue

### Method 1: Querying the Index Directly

This seems to have yielded the best results so far. It reliably knows what to do with the prompt and when prodded, it will output the entire file. It's not perfect, but it's about what i would expect of chatgpt. 

However it's SLOW AS FUCK and seems to consume a lot of tokens on every call. Not ideal.

In [34]:
from llama_index import QueryMode
resp = gpt35t_index.query(f"{PROMPT}. Pretend you are actually updating the file yourself and respond with the updated contents of the file in its entirety. Do not use markdown.", verbose=True)
# logger.info(resp)
display(resp.response)


> Got node text: file_path: lambda.template.yaml
file_name: lambda.template.yaml

AWSTemplateFormatVersion: "2010-09-09"
Transform: AWS::Serverless-2016-10-31
Description: >
  tacostats v1.5.4

  periodically ...


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 6000 tokens


> [query] Total LLM token usage: 6000 tokens
> [query] Total LLM token usage: 6000 tokens
> [query] Total LLM token usage: 6000 tokens
> [query] Total LLM token usage: 6000 tokens


INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 51 tokens


> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens


'AWSTemplateFormatVersion: "2010-09-09"\r\nTransform: AWS::Serverless-2016-10-31\r\nDescription: >\r\n  tacostats v1.5.4\r\n\r\n  periodically grab comments from the r/neoliberal DT and gather some basic statistics\r\n\r\n# More info about Globals: https://github.com/awslabs/serverless-application-model/blob/master/docs/globals.rst\r\nGlobals:\r\n  Function:\r\n    Timeout: 900\r\n\r\nResources:\r\n  ###############\r\n  # Harvester\r\n  ###############\r\n  Harvester:\r\n    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction\r\n    Properties:\r\n      PackageType: Image\r\n      MemorySize: 512 # Updated memory size\r\n      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4\r\n      ImageConfig:\r\n        Command:\r\n          - tacostats.harvester.lambda_handler\r\n      Role: arn:aws:iam::390721581096:role/tacostats\r\n      Event

### Method 2: Using langchain's GPTIndex tools

The general idea here is to provide a chat agent with a set of "tools", ie: our file index. 

This is supposed to be the best way to do it as it would behave more like ChatGPT's plugin system, but as you'll see, it doesn't actually work that well.

The best result that I got consistently was actually two separate responses. The first would say "You can update the memorysize by blahblahblah". When prompted a second time with "cool but just do it k?" it would flip between "but i'm just an LLM i can't actually write files" and outputting snippets in code blocks (despite the prompt specifically requesting it not to). 

Results were same or worse with different memory types, response types, and prompt templates.

I suspect that some of langchain's abstraction magic is getting in my way here but I'm not sure how to get it to behave the way I'd expect it to.

In [45]:

from dataclasses import asdict
from langchain.agents import Tool
from langchain.agents.agent_types import AgentType
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory
# from langchain.schema import ChatMessage

### it really didn't react well to having both an index memory like this and an index tool at the same time...
# gpt35t_memory_index = GPTListIndex([])
# gpt35t_memory = GPTIndexChatMemory(
#     index=gpt35t_memory_index, 
#     memory_key="chat_history", 
#     query_kwargs={"response_mode": "compact"},
#     # return_source returns source nodes instead of querying index
#     return_source=True,
#     # return_messages returns context in message format
#     return_messages=True
# )
gpt35t_memory = ConversationBufferMemory(memory_key="chat_history")
gpt35t_tools = [
        Tool(
            name = "GPT Index",
            func=lambda q: str(gpt35t_index.query(q)),
            description="these are the files you will need to update.",
            return_direct=True,
        )
    ]
gpt35t_model = ChatOpenAI(
    **{k: v for k, v in asdict(gpt35t_model_opts).items() if v},
)
gpt35t_agent_chain = initialize_agent(
    gpt35t_tools, 
    gpt35t_model, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    memory=gpt35t_memory,
    prompt_template=PROMPT_TEMPLATE,
    verbose=True,
)


In [ ]:
gpt35t_agent_chain.run(input=PROMPT)

In [ ]:
gpt35t_agent_chain.run(input="that's great now output the updated file in its entirety as a code block")

### Results

Well, gpt-3.5-turbo definitely updated the memory size, but it missed the "harvester-related" part and updated *all* functions. In other runs it bumped up MemorySize seemingly at random. Once it also added MemorySize to the Function globals. While I didn't run them to check, all of the generated code seemed to at least be syntactically correct even they didn't meet the requirements.

# gpt-4

## Options

In [61]:
# this goes into the service context and it seems like this cannot be changed after the index is created
gpt4_model_opts = ModelOptions(model_name="gpt-4", temperature=0.2, request_timeout=600)


## Create index

In [62]:
from alchemy.loader import create_simple_vector_index
gpt4_index = create_simple_vector_index(repo_opts, cache_opts, gpt4_model_opts)

INFO:alchemy.loader:loading simple vector index...


loading simple vector index...
loading simple vector index...
loading simple vector index...
loading simple vector index...
loading simple vector index...
loading simple vector index...


INFO:alchemy.loader:repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}


repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_modules'], <FilterType.EXCLUDE: 1>), 'filter_file_extensions': (['.zip'], <FilterType.EXCLUDE: 1>), 'verbose': True, 'concurrent_requests': 10}
repo_opts: {'owner': 'inhumantsar', 'repo': 'tacostats', 'branch': 'main', 'commit_sha': None, 'filter_directories': (['.git', '.yarn', 'node_

INFO:alchemy.loader:cache_opts: {'path': '~/.alchemy', 'force_update': True, 'max_age': 60}


...

model_opts: {'cache': None, 'verbose': False, 'model_name': 'gpt-4', 'temperature': 0.2, 'max_tokens': None, 'top_p': None, 'frequency_penalty': None, 'presence_penalty': None, 'n': None, 'best_of': None, 'model_kwargs': None, 'openai_api_key': None, 'batch_size': None, 'request_timeout': 600, 'logit_bias': None, 'max_retries': None, 'streaming': False}
model_opts: {'cache': None, 'verbose': False, 'model_name': 'gpt-4', 'temperature': 0.2, 'max_tokens': None, 'top_p': None, 'frequency_penalty': None, 'presence_penalty': None, 'n': None, 'best_of': None, 'model_kwargs': None, 'openai_api_key': None, 'batch_size': None, 'request_timeout': 600, 'logit_bias': None, 'max_retries': None, 'streaming': False}
model_opts: {'cache': None, 'verbose': False, 'model_name': 'gpt-4', 'temperature': 0.2, 'max_tokens': None, 'top_p': None, 'frequency_penalty': None, 'presence_penalty': None, 'n': None, 'best_of': None, 'model_kwargs': None, 'openai_api_key': None, 'batch_size': None, 'request_timeout'

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens


> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 32893 tokens


> [build_index_from_nodes] Total embedding token usage: 32893 tokens
> [build_index_from_nodes] Total embedding token usage: 32893 tokens
> [build_index_from_nodes] Total embedding token usage: 32893 tokens
> [build_index_from_nodes] Total embedding token usage: 32893 tokens
> [build_index_from_nodes] Total embedding token usage: 32893 tokens
> [build_index_from_nodes] Total embedding token usage: 32893 tokens


## Simulate a GitHub issue

In [65]:
from llama_index import QueryMode


gpt4_resp = gpt4_index.query(f"{PROMPT}. Pretend you are actually updating the file yourself and respond with the updated contents of the file in its entirety. Do not use markdown.", verbose=True)
# logger.info(resp)
display(gpt4_resp.response)

> Got node text: file_path: lambda.template.yaml
file_name: lambda.template.yaml

AWSTemplateFormatVersion: "2010-09-09"
Transform: AWS::Serverless-2016-10-31
Description: >
  tacostats v1.5.4

  periodically ...


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 5793 tokens


> [query] Total LLM token usage: 5793 tokens
> [query] Total LLM token usage: 5793 tokens
> [query] Total LLM token usage: 5793 tokens
> [query] Total LLM token usage: 5793 tokens
> [query] Total LLM token usage: 5793 tokens
> [query] Total LLM token usage: 5793 tokens


INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 51 tokens


> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens
> [query] Total embedding token usage: 51 tokens


'AWSTemplateFormatVersion: "2010-09-09"\nTransform: AWS::Serverless-2016-10-31\nDescription: >\n  tacostats v1.5.4\n\n  periodically grab comments from the r/neoliberal DT and gather some basic statistics\n\n# More info about Globals: https://github.com/awslabs/serverless-application-model/blob/master/docs/globals.rst\nGlobals:\n  Function:\n    Timeout: 900\n\nResources:\n  ###############\n  # Harvester\n  ###############\n  Harvester:\n    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction\n    Properties:\n      PackageType: Image\n      MemorySize: 512\n      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4\n      ImageConfig:\n        Command:\n          - tacostats.harvester.lambda_handler\n      Role: arn:aws:iam::390721581096:role/tacostats\n      Events:\n        Cron:\n          Type: Schedule\n          Properties:\n      

### Results

GPT-4 definitely did the thing. However, gpt-3.5-turbo came back with he exact same output in less time. It might be that GPT4 is more reliable or it might be that llama-index is actually just not that good. I feel like I get better results from GPT4 in chatgpt.

In the output below, note that it *didn't* create a new Globals entry for MemorySize, which is something GPT3 had a habit of doing, but it *did* update UserStats for some reason, which is also something GPT3 had a habit of doing...

```yaml
AWSTemplateFormatVersion: "2010-09-09"
Transform: AWS::Serverless-2016-10-31
Description: >
  tacostats v1.5.4

  periodically grab comments from the r/neoliberal DT and gather some basic statistics

# More info about Globals: https://github.com/awslabs/serverless-application-model/blob/master/docs/globals.rst
Globals:
  Function:
    Timeout: 900

Resources:
  ###############
  # Harvester
  ###############
  Harvester:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      PackageType: Image
      MemorySize: 512
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.harvester.lambda_handler
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(55/30 * * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

  # post a recap of yesterday\'s stats in the morning
  HarvesterRecap:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      PackageType: Image
      MemorySize: 512
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.harvester.lambda_handler
      Environment:
        Variables:
          RECAP: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 7 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

  ###############
  # Statistics
  ###############
  Stats:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      PackageType: Image
      MemorySize: 256
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.stats.lambda_handler
      Role: arn:aws:iam::390721581096:role/tacostats
      Environment:
        Variables:
          USE_EXISTING: True
      Events:
        # schedules in utc, et equivs: 12,16,21
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 16,22,1 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

  # post a recap of yesterday\'s stats in the morning
  StatsRecap:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      PackageType: Image
      MemorySize: 256
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.stats.lambda_handler
      Environment:
        Variables:
          RECAP: True
          USE_EXISTING: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        # schedules in utc, 9et
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 13 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

  ###############
  # Keywords
  ###############
  Keywords:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      PackageType: Image
      MemorySize: 384
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.keywords.lambda_handler
      Environment:
        Variables:
          USE_EXISTING: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(20 20 * * ? *) # et equiv: 16:20
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

  # post a recap of yesterday\'s keywords in the morning
  KeywordsRecap:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      PackageType: Image
      MemorySize: 384
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.keywords.lambda_handler
      Environment:
        Variables:
          RECAP: True
          USE_EXISTING: True
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        # schedules in utc, 10et
        Cron:
          Type: Schedule
          Properties:
            Schedule: cron(0 14 * * ? *)
            Enabled: True
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."

  ###############
  # UserStats
  ###############
  UserStats:
    Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
    Properties:
      Timeout: 300
      PackageType: Image
      MemorySize: 512
      ImageUri: 390721581096.dkr.ecr.us-east-2.amazonaws.com/tacostats:v1.5.4
      ImageConfig:
        Command:
          - tacostats.userstats.lambda_handler
      Role: arn:aws:iam::390721581096:role/tacostats
      Events:
        Queue:
          Type: SQS
          Properties:
            Queue: arn:aws:sqs:us-east-2:390721581096:tacostats-pinger
            BatchSize: 10
            Enabled: true
    Metadata:
      Dockerfile: lambda.Dockerfile
      DockerContext: "."
```